In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point

In [2]:
# 한글 폰트 문제 해결 
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

In [3]:
earns = pd.read_csv("Data/점포별평균매출2020_2024.csv", index_col=0)
ingu =  pd.read_csv('Data/나이대별_인구수.csv')
geo_data_hdong = gpd.read_file("Data/서울시 상권분석서비스(영역-행정동)/서울시 상권분석서비스(영역-행정동).shp")
geo_data_hdong['ADSTRD_NM'] = geo_data_hdong['ADSTRD_NM'].map(lambda x: x.replace('·', '?'))

df_list  = []
df_list.append(pd.read_csv('Data/gs25_only_latlng.csv').loc[:, ['lat', 'lng']])
df_list.append(pd.read_csv('Data/Cu_unique.csv').loc[:, ['lat', 'lng']])
df_list.append(pd.read_csv('Data/seven_reduced.csv').loc[:, ['lat', 'lng']])
df_list.append(pd.read_csv('Data/emart_only_latlng.csv').loc[:, ['lat', 'lng']])
convs = pd.concat(df_list, axis=0)

In [4]:
earns_20230 = earns[(earns['기준_년분기_코드'] > 20200) & (earns['서비스_업종_코드_명'] == '편의점')]
earns_20230

,기준_년분기_코드,행정동_코드_명,서비스_업종_코드_명,당월_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액
0,20201,상계5동,편의점,7.231475e+07,5.197630e+05,1.734100e+07,1.648128e+07,1.727974e+07,1.367176e+07,7.021210e+06
1,20201,보문동,편의점,6.395841e+07,7.946551e+05,2.321184e+07,1.674104e+07,1.269793e+07,6.646649e+06,3.866299e+06
2,20201,청량리동,편의점,7.949244e+07,3.662401e+05,1.862711e+07,2.258412e+07,1.975032e+07,1.309739e+07,5.067265e+06
3,20201,답십리1동,편의점,5.665572e+07,6.445178e+05,1.542066e+07,1.700094e+07,1.303524e+07,7.458664e+06,3.095698e+06
4,20201,행당2동,편의점,1.172458e+08,4.057786e+06,2.933915e+07,3.080714e+07,2.725581e+07,1.714016e+07,8.645717e+06
...,...,...,...,...,...,...,...,...,...,...
7476,20242,성내1동,편의점,1.260640e+08,7.515665e+05,1.678108e+07,3.662918e+07,3.924313e+07,2.192998e+07,1.072904e+07
7477,20242,성내2동,편의점,2.198670e+08,1.636622e+06,4.257493e+07,7.183623e+07,4.477765e+07,3.394683e+07,2.509473e+07
7478,20242,성내3동,편의점,1.130876e+08,1.446543e+06,1.774175e+07,2.980714e+07,2.740294e+07,2.335457e+07,1.333462e+07
7479,20242,길동,편의점,1.685806e+08,8.579869e+05,2.462767e+07,4.863248e+07,4.145572e+07,3.163814e+07,2.136861e+07


In [5]:
new_earns = pd.merge(earns_20230, ingu).iloc[:, :]
new_earns =  new_earns.drop('행정동_코드', axis = 1)


In [6]:
def point_buffer_func(lng, lat, radios = 500): ### radius
    point = Point(lng, lat)

    # GeoDataFrame 생성 및 위경도 좌표계(EPSG:4326) 설정
    gdf_point = gpd.GeoDataFrame(geometry=[point], crs="EPSG:4326")

    # 한국의 평면 좌표계 (예: EPSG:5191)로 변환
    gdf_point = gdf_point.to_crs(epsg=5181)

    # 200m 반경 버퍼 생성
    gdf_point['buffer_200m'] = gdf_point.geometry.buffer(radios)
    point_buffer =  gdf_point.iloc[:,[-1]]
    point_buffer.columns = ['geometry']

    return point_buffer

In [7]:
def point_ratio_func(geo_data, point_buffer):
    geo_data_converted = geo_data.to_crs(epsg=5181)
    gdf_intersection = gpd.overlay(geo_data_converted, point_buffer, how='intersection')
    gdf_intersection['ratio'] = gdf_intersection.geometry.area / point_buffer.geometry.area.item()
    point_ratio = pd.DataFrame(gdf_intersection.iloc[:, [1, -1]], )
    point_ratio.set_index(point_ratio.columns[0], inplace=True)
    point_ratio.loc['기타', :] = (1 -  point_ratio.iloc[:, 0].sum())

    return point_ratio

In [8]:
def ratio_earned_func(earns, geo_data, lng, lat, wanted_yaer):

    point_buffer = point_buffer_func(lng, lat)



    convs = earns[(earns['서비스_업종_코드_명'] == '편의점') & (earns['기준_년분기_코드'] == wanted_yaer)]
    connvs_hdong = convs.pivot_table(index = '행정동_코드_명', values = convs.columns[4:], aggfunc='mean') ## 이부분

    A = pd.Series(geo_data['ADSTRD_NM'].unique())
    B = pd.Series(connvs_hdong.index.unique())
    result1 = A[~A.isin(B)]
    result2 = B[~B.isin(A)]

    nan_geo = pd.concat([result1, result2], axis= 0 ).unique()

    geo_data2 = geo_data.drop(geo_data[geo_data['ADSTRD_NM'].isin(nan_geo)].index)
    point_ratio = point_ratio_func(geo_data2, point_buffer)

    connvs_hdong.loc['기타', :] = 1



    connvs_hdong_reduced = connvs_hdong.loc[list(point_ratio.index), :]
    connvs_hdong_reduced.loc['기타', :] = connvs_hdong_reduced.iloc[:].mean()
    ratio_earned = connvs_hdong_reduced.mul(point_ratio['ratio'], axis=0).sum(axis=0)



    
    return ratio_earned

In [9]:
def ratio_ingu_func(ingu, geo_data, lng, lat, wanted_yaer):

    point_buffer = point_buffer_func(lng, lat)


    convs = ingu[(ingu['기준_년분기_코드'] == wanted_yaer)]
    connvs_hdong = convs.pivot_table(index = '행정동_코드_명', values =ingu.columns[3:], aggfunc='mean')


    A = pd.Series(geo_data['ADSTRD_NM'].unique())
    B = pd.Series(connvs_hdong.index.unique())
    result1 = A[~A.isin(B)]
    result2 = B[~B.isin(A)]


    nan_geo = pd.concat([result1, result2], axis= 0 ).unique()

    geo_data2 = geo_data.drop(geo_data[geo_data['ADSTRD_NM'].isin(nan_geo)].index)

    connvs_hdong.loc['기타', :] = 1

    point_ratio = point_ratio_func(geo_data2, point_buffer)
    connvs_hdong_reduced = connvs_hdong.loc[list(point_ratio.index), :]
    connvs_hdong_reduced.loc['기타', :] = connvs_hdong_reduced.iloc[:].mean()
    ratio_earned = connvs_hdong_reduced.mul(point_ratio['ratio'], axis=0).sum(axis=0)
    ratio_earned
    
    return ratio_earned

In [10]:
store_money_list = []
quater = 20204
for i, (lat, lng) in enumerate( zip(convs['lat'], convs['lng'])):
    try:
        store_money_list.append(ratio_earned_func(new_earns, geo_data_hdong, lng, lat, wanted_yaer = quater))
    except Exception as e:
        print(e)
        store_money_list.append(np.nan)
    
    if i % 100 == 0:
        print(f'{i}번째 완료')
    # if i == 100:
    #     break
        
    if i %1000 == 999:
        print({'--------------------------------------'})

pd.concat([convs.reset_index(drop=True), pd.DataFrame(store_money_list)], axis=1).to_csv(f'data_v2_{quater}.csv')

0번째 완료
100번째 완료
200번째 완료
300번째 완료
400번째 완료
500번째 완료
600번째 완료
700번째 완료
800번째 완료
900번째 완료
{'--------------------------------------'}
1000번째 완료
1100번째 완료
1200번째 완료
1300번째 완료
1400번째 완료
1500번째 완료


In [11]:
store_money_list = []
quater = 20214
for i, (lat, lng) in enumerate( zip(convs['lat'], convs['lng'])):
    try:
        store_money_list.append(ratio_earned_func(new_earns, geo_data_hdong, lng, lat, wanted_yaer = quater))
    except Exception as e:
        print(e)
        store_money_list.append(np.nan)
    
    if i % 100 == 0:
        print(f'{i}번째 완료')
    # if i == 100:
    #     break
        
    if i %1000 == 999:
        print({'--------------------------------------'})

pd.concat([convs.reset_index(drop=True), pd.DataFrame(store_money_list)], axis=1).to_csv(f'data_v2_{quater}.csv')

0번째 완료
100번째 완료
200번째 완료
300번째 완료
400번째 완료
500번째 완료
600번째 완료
700번째 완료
800번째 완료
900번째 완료
{'--------------------------------------'}
1000번째 완료
1100번째 완료
1200번째 완료
1300번째 완료
1400번째 완료
1500번째 완료
1600번째 완료
1700번째 완료
1800번째 완료
1900번째 완료
{'--------------------------------------'}
2000번째 완료
2100번째 완료
2200번째 완료
2300번째 완료
2400번째 완료
2500번째 완료
2600번째 완료
2700번째 완료
2800번째 완료
2900번째 완료
{'--------------------------------------'}
3000번째 완료
3100번째 완료
3200번째 완료
3300번째 완료
3400번째 완료
3500번째 완료
3600번째 완료
3700번째 완료
3800번째 완료
3900번째 완료
{'--------------------------------------'}
4000번째 완료
4100번째 완료
4200번째 완료
4300번째 완료
4400번째 완료
4500번째 완료
4600번째 완료
4700번째 완료
4800번째 완료
4900번째 완료
{'--------------------------------------'}
5000번째 완료
5100번째 완료
5200번째 완료
5300번째 완료
5400번째 완료
5500번째 완료
5600번째 완료
5700번째 완료
5800번째 완료
5900번째 완료
{'--------------------------------------'}
6000번째 완료
6100번째 완료
6200번째 완료
6300번째 완료
6400번째 완료
6500번째 완료
6600번째 완료
6700번째 완료
6800번째 완료
6900번째 완료
{'--------------------------------------'}
7000번째 완료
7

In [12]:
store_money_list = []
quater = 20224
for i, (lat, lng) in enumerate( zip(convs['lat'], convs['lng'])):
    try:
        store_money_list.append(ratio_earned_func(new_earns, geo_data_hdong, lng, lat, wanted_yaer = quater))
    except Exception as e:
        print(e)
        store_money_list.append(np.nan)
    
    if i % 100 == 0:
        print(f'{i}번째 완료')
    # if i == 100:
    #     break
        
    if i %1000 == 999:
        print({'--------------------------------------'})

pd.concat([convs.reset_index(drop=True), pd.DataFrame(store_money_list)], axis=1).to_csv(f'data_v2_{quater}.csv')

0번째 완료
100번째 완료
200번째 완료
300번째 완료
400번째 완료
500번째 완료
600번째 완료
700번째 완료
800번째 완료
900번째 완료
{'--------------------------------------'}
1000번째 완료
1100번째 완료
1200번째 완료
1300번째 완료
1400번째 완료
1500번째 완료
1600번째 완료
1700번째 완료
1800번째 완료
1900번째 완료
{'--------------------------------------'}
2000번째 완료
2100번째 완료
2200번째 완료
2300번째 완료
2400번째 완료
2500번째 완료
2600번째 완료
2700번째 완료
2800번째 완료
2900번째 완료
{'--------------------------------------'}
3000번째 완료
3100번째 완료
3200번째 완료
3300번째 완료
3400번째 완료
3500번째 완료
3600번째 완료
3700번째 완료
3800번째 완료
3900번째 완료
{'--------------------------------------'}
4000번째 완료
4100번째 완료
4200번째 완료
4300번째 완료
4400번째 완료
4500번째 완료
4600번째 완료
4700번째 완료
4800번째 완료
4900번째 완료
{'--------------------------------------'}
5000번째 완료
5100번째 완료
5200번째 완료
5300번째 완료
5400번째 완료
5500번째 완료
5600번째 완료
5700번째 완료
5800번째 완료
5900번째 완료
{'--------------------------------------'}
6000번째 완료
6100번째 완료
6200번째 완료
6300번째 완료
6400번째 완료
6500번째 완료
6600번째 완료
6700번째 완료
6800번째 완료
6900번째 완료
{'--------------------------------------'}
7000번째 완료
7

In [13]:
store_money_list = []
quater = 20234
for i, (lat, lng) in enumerate( zip(convs['lat'], convs['lng'])):
    try:
        store_money_list.append(ratio_earned_func(new_earns, geo_data_hdong, lng, lat, wanted_yaer = quater))
    except Exception as e:
        print(e)
        store_money_list.append(np.nan)
    
    if i % 100 == 0:
        print(f'{i}번째 완료')
    # if i == 100:
    #     break
        
    if i %1000 == 999:
        print({'--------------------------------------'})

pd.concat([convs.reset_index(drop=True), pd.DataFrame(store_money_list)], axis=1).to_csv(f'data_v2_{quater}.csv')

0번째 완료
100번째 완료
200번째 완료
300번째 완료
400번째 완료
500번째 완료
600번째 완료
700번째 완료
800번째 완료
900번째 완료
{'--------------------------------------'}
1000번째 완료
1100번째 완료
1200번째 완료
1300번째 완료
1400번째 완료
1500번째 완료
1600번째 완료
1700번째 완료
1800번째 완료
1900번째 완료
{'--------------------------------------'}
2000번째 완료
2100번째 완료
2200번째 완료
2300번째 완료
2400번째 완료
2500번째 완료
2600번째 완료
2700번째 완료
2800번째 완료
2900번째 완료
{'--------------------------------------'}
3000번째 완료
3100번째 완료
3200번째 완료
3300번째 완료
3400번째 완료
3500번째 완료
3600번째 완료
3700번째 완료
3800번째 완료
3900번째 완료
{'--------------------------------------'}
4000번째 완료
4100번째 완료
4200번째 완료
4300번째 완료
4400번째 완료
4500번째 완료
4600번째 완료
4700번째 완료
4800번째 완료
4900번째 완료
{'--------------------------------------'}
5000번째 완료
5100번째 완료
5200번째 완료
5300번째 완료
5400번째 완료
5500번째 완료
5600번째 완료
5700번째 완료
5800번째 완료
5900번째 완료
{'--------------------------------------'}
6000번째 완료
6100번째 완료
6200번째 완료
6300번째 완료
6400번째 완료
6500번째 완료
6600번째 완료
6700번째 완료
6800번째 완료
6900번째 완료
{'--------------------------------------'}
7000번째 완료
7

In [81]:
store_money_list[1]


10대_상주인구수          2.920575e+03
10대_유동인구수          8.004487e+05
10대_인구수            8.037592e+05
10대_직장인구수          3.901254e+02
20대_상주인구수          3.026931e+03
20대_유동인구수          3.383137e+05
20대_인구수            3.572042e+05
20대_직장인구수          1.586373e+04
30대_상주인구수          2.183097e+03
30대_유동인구수          4.230015e+05
30대_인구수            4.575424e+05
30대_직장인구수          3.235804e+04
40대_상주인구수          2.844639e+03
40대_유동인구수          6.187737e+05
40대_인구수            6.455811e+05
40대_직장인구수          2.396287e+04
50대_상주인구수          3.913960e+03
50대_유동인구수          4.917784e+05
50대_인구수            5.049164e+05
50대_직장인구수          9.224232e+03
60대_이상_상주인구수       4.150986e+03
60대_이상_유동인구수       5.893983e+05
60대_이상_직장인구수       2.216365e+03
60대_인구수            5.957655e+05
연령대_20_매출_금액       1.537381e+07
연령대_30_매출_금액       2.618771e+07
연령대_40_매출_금액       3.152395e+07
연령대_50_매출_금액       2.514460e+07
연령대_60_이상_매출_금액    1.280312e+07
dtype: float64

In [52]:
ingu_series_list = []
quater = 20231
for i, (lat, lng) in enumerate( zip(convs['lat'], convs['lng'])):
    try:
        ingu_series_list.append(ratio_ingu_func(ingu, geo_data_hdong, lng, lat, wanted_yaer = quater))
    except Exception as e:
        print(e)
        ingu_series_list.append(np.nan)
    
    if i % 100 == 0:
        print(f'{i}번째 완료')
    if i % 1000 == 999:
        print('-' * 30)

ingu_ratio = pd.DataFrame(
    ingu_series_list
)

ingu_ratio.to_csv(f'../data/ingu_{quater}.csv')

0번째 완료
100번째 완료
200번째 완료
300번째 완료
400번째 완료
500번째 완료
600번째 완료
700번째 완료
800번째 완료
900번째 완료
------------------------------
1000번째 완료
1100번째 완료
1200번째 완료
1300번째 완료
1400번째 완료
1500번째 완료
1600번째 완료
1700번째 완료
1800번째 완료
1900번째 완료
------------------------------
2000번째 완료
2100번째 완료
2200번째 완료
2300번째 완료
2400번째 완료
2500번째 완료
2600번째 완료
2700번째 완료
2800번째 완료
2900번째 완료
------------------------------
3000번째 완료
3100번째 완료
3200번째 완료
3300번째 완료
3400번째 완료
3500번째 완료
3600번째 완료
3700번째 완료
3800번째 완료
3900번째 완료
------------------------------
4000번째 완료
4100번째 완료
4200번째 완료
4300번째 완료
4400번째 완료
4500번째 완료
4600번째 완료
4700번째 완료
4800번째 완료
4900번째 완료
------------------------------
5000번째 완료
5100번째 완료
5200번째 완료
5300번째 완료
5400번째 완료
5500번째 완료
5600번째 완료
5700번째 완료
5800번째 완료
5900번째 완료
------------------------------
6000번째 완료
6100번째 완료
6200번째 완료
6300번째 완료
6400번째 완료
6500번째 완료
6600번째 완료
6700번째 완료
6800번째 완료
6900번째 완료
------------------------------
7000번째 완료
7100번째 완료
7200번째 완료
7300번째 완료
7400번째 완료
7500번째 완료
7600번째 완료
7700번째 완료
7800번째 완료
7900번

In [13]:
quaters = [20231, 20232, 20233, 20234, 20241, 20242]

In [53]:
ingu_ratio = pd.DataFrame(
    ingu_series_list
)

ingu_ratio.to_csv(f'../data/ingu_{quater}.csv')

In [16]:
for quater in quaters:
    ingu = pd.read_csv(f'../data/ingu_{quater}.csv').iloc[:, 1:]
    money = pd.read_csv(f'../data/money_target_{quater}.csv').iloc[:, 1:]
    pd.concat([money, ingu], axis=1).to_csv(f'../data/train_data{quater}.csv')

In [12]:
pd.concat([money, ingu], axis=1)

,lat,lng,money,10대_상주인구수,10대_유동인구수,10대_직장인구수,20대_상주인구수,20대_유동인구수,20대_직장인구수,30대_상주인구수,...,30대_직장인구수,40대_상주인구수,40대_유동인구수,40대_직장인구수,50대_상주인구수,50대_유동인구수,50대_직장인구수,60대_이상_상주인구수,60대_이상_유동인구수,60대_이상_직장인구수
0,37.479196,127.045336,1.303922e+08,4067.899349,8.315558e+05,259.139224,3018.700106,6.636712e+05,8529.316960,3897.217756,...,15703.396342,4813.994915,9.818022e+05,12982.363938,4352.364278,7.788781e+05,7099.129369,6385.529845,9.491025e+05,2466.014905
1,37.484878,127.054228,9.840353e+07,2264.056137,5.168492e+05,338.588461,2325.327852,2.470825e+05,13567.984504,1783.466977,...,27373.134457,2212.654303,4.608290e+05,20683.861111,2981.435549,3.782054e+05,8596.229941,3514.859157,4.728712e+05,2379.101123
2,37.477946,127.047278,1.307643e+08,2841.208175,6.711320e+05,108.467874,2518.349648,5.781983e+05,4281.170189,3393.475351,...,7972.043689,3801.400793,8.226435e+05,7649.129950,3634.524584,7.008219e+05,5438.270201,5193.685424,8.134521e+05,1955.566376
3,37.478860,127.049296,1.316636e+08,2340.213952,6.013396e+05,74.072896,2432.898120,4.621108e+05,3255.180530,2866.951370,...,6421.587144,3160.445783,6.578094e+05,5925.660863,3377.880150,6.153482e+05,3732.050806,4587.255066,7.098855e+05,1459.011909
4,37.476060,127.047090,1.299559e+08,2622.501588,6.469968e+05,65.917513,2323.309723,6.380094e+05,3290.719659,3594.635109,...,5994.961995,3825.341069,8.887779e+05,6943.361849,3478.948548,7.362046e+05,6363.295716,5071.119749,8.324327e+05,2172.754586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8494,37.508244,127.033112,2.154536e+08,2007.515279,1.022990e+06,274.096010,2249.341615,3.353926e+06,18978.292805,6955.527483,...,27267.704223,6083.521734,2.943282e+06,22405.384307,3693.212799,1.707105e+06,15068.941382,6011.430983,1.710396e+06,6650.101822
8495,37.503377,127.113663,9.024796e+07,2783.186529,7.492382e+05,41.458610,2825.068990,6.806870e+05,1716.717729,4156.886282,...,2740.038015,3887.911975,7.133671e+05,2947.765088,3666.339047,6.102555e+05,2661.209221,6208.926881,8.932663e+05,1371.455663
8496,37.477042,126.881968,1.859301e+08,1421.395456,2.337646e+05,298.824854,1461.238246,1.152006e+06,16575.600553,5053.065746,...,26124.921197,3179.458556,9.126768e+05,20807.900899,2685.407962,7.019313e+05,12320.390604,4837.914681,7.093222e+05,7991.471493
8497,37.557077,126.922527,1.246287e+08,1849.430995,1.750628e+06,110.411297,2169.929798,5.365404e+06,4666.757680,5805.996213,...,6296.694214,4441.865749,2.313472e+06,5224.071310,3153.053316,1.519907e+06,3705.463241,5675.320844,1.728103e+06,2504.416033
